# Spark Streaming Estructurado

En este ejercicio de Streaming veremos un escenario más cercano a un entorno real, en el que generaremos datos estructurados para poder ser explotados a través de consultas SQL 

Generamos desde el Shell el directorio /structstream con un fichero devdata.txt que tengo el contenido que tienes abajo
hdfs dfs -mkdir /structstream

nano devdata.txt

{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}

Ahora, definiremos un objeto consulta que escribe la salida a una tabla en memoria llamada counts. Esta técnica es útil para testea de un programa de streaming estructurado, pero en una solución de producción se almacenario en un fichero de salida o una base de datos. 

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
inputPath = "wasb:///structstream/"
jsonSchema = StructType([
StructField("device", StringType(), False),
StructField("status", StringType(), False)
])
fileDF = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)
countDF = fileDF.filter("status == 'error'").groupBy("device").count()
query = countDF.writeStream.format("memory").queryName("counts").outputMode("complete").start()

Al igual que hicimos en el ejercicio anterior, generamos ficheros para que puedan ser leídos por el Stream

hdfs dfs -put devdata.txt /structstream/1
hdfs dfs -put devdata.txt /structstream/2


In [ ]:
Consultamos la tabla en memoria

In [ ]:
%%sql
select * from counts

Generamos otro fichero
hdfs dfs -put devdata.txt /structstream/3

consultamos de nuevo

In [ ]:
%%sql
select * from counts

In [ ]:
Detenemos la consulta

In [ ]:
query.stop()